![Globe](Globe.jpg)


# [ ![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg) ](https://nbviewer.org/github/alisonnanjez/World-Indicators-DB/blob/main/notebook.ipynb)

## Introduction

This notebook explores global development indicators, including economic factors, education levels, health metrics, and labor force participation, to uncover potential relationships and regional variations across countries. The analysis leverages a comprehensive dataset sourced from [Kaggle]( https://www.kaggle.com/datasets/arslaan5/global-data-gdp-life-expectancy-and-more) which compiles a wide array of socio-economic data points at the country level. By structuring this data into a relational database and employing SQL queries, we aim to extract meaningful insights into the interconnectedness of these global development aspects.

## The Data
The dataset for this analysis, "Global Data: GDP, Life Expectancy & More," available on Kaggle, offers a comprehensive snapshot of key economic and social indicators across 204 entries and 38 distinct attributes. This rich collection allows for an in-depth exploration of global trends related to crucial aspects of development, including economic output (GDP), demographic structures (Sex Ratio, Urban Population Growth), health (Life Expectancy, Homicide Rate, Infant Mortality), education (Enrollment Rates across various levels), labor (Unemployment Rate), and environmental factors (CO2 Emissions, Forested Area, Tourist Numbers). The breadth and depth of these attributes provide a solid foundation for uncovering meaningful patterns and relationships across the globe.

## From Raw Data to Relational Database

Prior to analysis, the initial dataset underwent a cleaning process to handle inconsistencies and missing values, ensuring data integrity for subsequent steps. For the purpose of focused analysis, the comprehensive dataset was then logically subdivided into multiple CSV files, each representing a specific aspect of global development: demographics, economy, education, labour, and lifespan. These CSV files were subsequently converted into corresponding tables within a SQLite database named `world_data_v4.db`. The database schema was designed with these five tables as primary dimensions, each containing relevant indicators and linked by a common `country_code` identifier to facilitate cross-dimensional analysis and the creation of a central `country_facts` table for integrated insights.

In [ ]:
import pandas as pd
import sqlite3
import os

def process_and_load_csvs_to_sql_with_duplicates_dropped(csv_paths, db_name, table_names, subset_columns=None):
    """
    Reads specific CSV files from provided paths, drops rows with missing values AND duplicates,
    and loads each processed DataFrame into a SQLite database with specified table names.

    Args:
        csv_paths (list): A list of strings, where each string is the full path to a CSV file.
        db_name (str): The name of the SQLite database file to create or connect to (e.g., 'world.db').
        table_names (list): A list of strings, where each string is the desired table name
                            in the database for the corresponding CSV file in csv_paths.
                            The order of table names should match the order of CSV paths.
        subset_columns (list, optional): A list of column names to consider when identifying
                                         duplicates. If None, all columns are considered.
                                         Defaults to None.
    """
    try:
        # Connect to the SQLite database (it will be created if it doesn't exist)
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        if len(csv_paths) != len(table_names):
            raise ValueError("The number of CSV paths must match the number of table names.")

        for i, csv_path in enumerate(csv_paths):
            table_name = table_names[i]
            print(f"Processing file: {csv_path} and loading into table: {table_name}")

            try:
                # Read the CSV file into a pandas DataFrame
                df = pd.read_csv(csv_path)
                print(f"  Shape before dropping NaNs: {df.shape}")

                # Drop rows with any missing values
                df_cleaned = df.dropna()
                print(f"  Shape after dropping NaNs: {df_cleaned.shape}")

                # Drop duplicate rows
                df_deduplicated = df_cleaned.drop_duplicates(subset=subset_columns)
                print(f"  Shape after dropping duplicates: {df_deduplicated.shape}")

                # Load the cleaned and deduplicated DataFrame into the SQLite database
                df_deduplicated.to_sql(table_name, conn, if_exists='replace', index=False)
                print(f"  DataFrame loaded into table: {table_name}")

            except FileNotFoundError:
                print(f"Error: File not found: {csv_path}")
            except pd.errors.EmptyDataError:
                print(f"Warning: File is empty: {file_path}")
            except Exception as e:
                print(f"An error occurred while processing {csv_path}: {e}")

        # Close the database connection
        if conn:
            conn.close()
            print(f"\nSuccessfully processed CSV files and loaded data into database: {db_name}")

    except ValueError as ve:
        print(f"Error: {ve}")
    except Exception as overall_e:
        print(f"An overall error occurred: {overall_e}")

# Example Usage (assuming you want to drop duplicates based on all columns):
if __name__ == "__main__":
    csv_file_paths = [
        'demographics.csv',
        'economy.csv',
        'education.csv',
        'labour.csv',
        'lifespan.csv'
    ]

    output_table_names = [
        'demographics',
        'economy',
        'education',
        'labour',
        'lifespan'
    ]

    database_name = 'world_data_v4.db'

    process_and_load_csvs_to_sql_with_duplicates_dropped(csv_file_paths, database_name, output_table_names)

    

Processing file: demographics.csv and loading into table: demographics
  Shape before dropping NaNs: (204, 7)
  Shape after dropping NaNs: (203, 7)
  Shape after dropping duplicates: (191, 7)
  DataFrame loaded into table: demographics
Processing file: economy.csv and loading into table: economy
  Shape before dropping NaNs: (204, 7)
  Shape after dropping NaNs: (198, 7)
  Shape after dropping duplicates: (198, 7)
  DataFrame loaded into table: economy
Processing file: education.csv and loading into table: education
  Shape before dropping NaNs: (204, 8)
  Shape after dropping NaNs: (167, 8)
  Shape after dropping duplicates: (167, 8)
  DataFrame loaded into table: education
Processing file: labour.csv and loading into table: labour
  Shape before dropping NaNs: (204, 6)
  Shape after dropping NaNs: (190, 6)
  Shape after dropping duplicates: (190, 6)
  DataFrame loaded into table: labour
Processing file: lifespan.csv and loading into table: lifespan
  Shape before dropping NaNs: (204,

## The SQLite Database Design

To interact with our SQLite database directly within this Jupyter Notebook, we utilized the `ipython-sql` extension. This powerful tool allows us to execute SQL queries and retrieve results seamlessly within our Python workflow. Before use, the extension was loaded using the `%load_ext sql` command, which registers the necessary magic commands. Subsequently, the line `%%sql sqlite:///world_data_v4.db` establishes a connection to our `world_data_v4.db` SQLite database, making its tables and data accessible for querying using SQL directly within code cells marked with the `%%sql` magic command. This integration streamlines the analytical process by enabling in-notebook database interactions.

In [ ]:
pip install ipython-sql

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 110.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sqlalchemy-redshift 0.8.11 requires SQLAlchemy<2.0.0,>=0.9.2, but you have sqlalchemy 2.0.40 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%load_ext sql

The code below shows the names of the tables in the world_data_v4 database

In [ ]:
%%sql sqlite:///world_data_v4.db
SELECT name
FROM sqlite_master
WHERE type='table';

Done.


name
demographics
economy
education
labour
lifespan
country_facts
sqlite_sequence


Each table within our `world_data_v4.db` database is structured with a designated primary key to ensure unique identification of records. The `demographics` table utilizes `country_code` as its primary key. The dimension tables for `economy` (`economy_id`), `education` (`education_id`), `labour` (`labour_id`), and `lifespan` (`lifespan_id`) each employ an auto-incrementing integer as their primary key. The process of creating these dimension tables involved defining their schema based on the corresponding CSV files, including appropriate data types for each attribute and designating the primary key. For the `demographics` table, this involved creating the table with `country_code` as the primary key. For the other dimension tables, this initially involved creating tables and loading data, followed by the addition of a unique, auto-incrementing primary key to facilitate efficient querying and linking within our database schema.

In [ ]:
%%sql

CREATE TABLE temp_demographics AS SELECT * FROM demographics;


DROP TABLE demographics;


CREATE TABLE demographics (
    country_code VARCHAR(2) PRIMARY KEY,
    name VARCHAR(255),
    region VARCHAR(100),
    urban_population REAL,
    pop_growth REAL,
    pop_density REAL,
    sex_ratio REAL
);


INSERT INTO demographics SELECT * FROM temp_demographics;

-- Drop the temporary table
DROP TABLE temp_demographics;

   sqlite:///world_data_v2.db
   sqlite:///world_data_v3.db
 * sqlite:///world_data_v4.db
Done.
Done.
Done.
191 rows affected.
Done.


[]

In [ ]:
%%sql


CREATE TABLE temp_economy AS SELECT * FROM economy;


DROP TABLE economy;


CREATE TABLE economy (
    economy_id INTEGER PRIMARY KEY,
    country_code VARCHAR(2),
    gdp REAL,
    imports REAL,
    exports REAL,
    gdp_growth REAL,
    gdp_per_capita REAL
);

INSERT INTO economy SELECT * FROM temp_economy;

-- Drop the temporary table
DROP TABLE temp_economy;


   sqlite:///world_data_v2.db
   sqlite:///world_data_v3.db
 * sqlite:///world_data_v4.db
Done.
Done.
Done.
198 rows affected.
Done.


[]

In [ ]:
%%sql

CREATE TABLE temp_education AS SELECT * FROM education;


DROP TABLE education;


CREATE TABLE education (
    education_id INTEGER PRIMARY KEY,
    country_code VARCHAR(2),
    prim_enrollment_female REAL,
    prim_enrollment_male REAL,
    sec_enrollment_female REAL,
    sec_enrollment_male REAL,
    post_sec_enrollment_female REAL,
    post_sec_enrollment_male REAL
);

INSERT INTO education SELECT * FROM temp_education;

-- Drop the temporary table
DROP TABLE temp_education;


   sqlite:///world_data_v2.db
   sqlite:///world_data_v3.db
 * sqlite:///world_data_v4.db
Done.
Done.
Done.
167 rows affected.
Done.


[]

In [ ]:
%%sql

CREATE TABLE temp_labour AS SELECT * FROM labour;


DROP TABLE labour;


CREATE TABLE labour (
    labour_id INTEGER PRIMARY KEY,
    country_code VARCHAR(2),
    employment_services REAL,
    employment_industry REAL,
    employment_agriculture REAL,
    unemployment REAL
);

INSERT INTO labour SELECT * FROM temp_labour;

-- Drop the temporary table
DROP TABLE temp_labour;

   sqlite:///world_data_v2.db
   sqlite:///world_data_v3.db
 * sqlite:///world_data_v4.db
Done.
Done.
Done.
190 rows affected.
Done.


[]

In [ ]:
%%sql

CREATE TABLE temp_lifespan AS SELECT * FROM lifespan;


DROP TABLE lifespan;


CREATE TABLE lifespan (
    lifespan_id INTEGER PRIMARY KEY,
    country_code VARCHAR (2),
    life_expectancy_female REAL,
    life_expectancy_male REAL,
    homicide_rate REAL,
    infant_mortality REAL,
    fertility REAL
);
INSERT INTO lifespan SELECT * FROM temp_lifespan;

-- Drop the temporary table
DROP TABLE temp_lifespan;

   sqlite:///world_data_v2.db
   sqlite:///world_data_v3.db
 * sqlite:///world_data_v4.db
Done.
Done.
Done.
172 rows affected.
Done.


[]

To facilitate integrated analysis across our different dimensions, a central `country_facts` table was created. This table acts as a linking mechanism, containing the `country_code` and the primary key identifiers from each of the dimension tables (`economy_id`, `education_id`, `labour_id`, and `lifespan_id`). The `country_facts` table was populated by performing `JOIN` operations across all the dimension tables based on the common `country_code`. This process allowed us to bring together related data points from different aspects of global development into single records within the `country_facts` table, enabling more complex queries and analyses that consider the interplay between economic, social, and health indicators for each country.

In [ ]:
%%sql
CREATE TABLE country_facts (
    fact_id INTEGER PRIMARY KEY AUTOINCREMENT,
    country_code VARCHAR(2),
    economy_id INTEGER,
    education_id INTEGER,
    labour_id INTEGER,
    lifespan_id INTEGER,
    FOREIGN KEY (country_code) REFERENCES demographics(country_code),
    FOREIGN KEY (economy_id) REFERENCES economy(economy_id),
    FOREIGN KEY (education_id) REFERENCES education(education_id),
    FOREIGN KEY (labour_id) REFERENCES labour(labour_id),
    FOREIGN KEY (lifespan_id) REFERENCES lifespan(lifespan_id)
);

   sqlite:///world_data_v2.db
   sqlite:///world_data_v3.db
 * sqlite:///world_data_v4.db
Done.


[]

In [ ]:
%%sql
INSERT INTO country_facts (country_code, economy_id, education_id, labour_id, lifespan_id)
SELECT
    d.country_code,
    e.economy_id,
    ed.education_id,
    l.labour_id,
    ls.lifespan_id
FROM
    demographics d
JOIN
    economy e ON d.country_code = e.country_code
JOIN
    education ed ON d.country_code = ed.country_code
JOIN
    labour l ON d.country_code = l.country_code
JOIN
    lifespan ls ON d.country_code = ls.country_code;

   sqlite:///world_data_v2.db
   sqlite:///world_data_v3.db
 * sqlite:///world_data_v4.db
271 rows affected.


[]

## Analysis in the World_data_v4 database

### GDP Growth Across Regions: Above or Below Global Trends?

The initial analysis investigated the variations in economic dynamism across different regions by examining their average GDP growth rates. To provide a global context, the overall average GDP growth across all countries in the dataset was calculated. By comparing the average GDP growth for each region to this global benchmark, it was possible to identify regions experiencing stronger or weaker economic expansion relative to the global trend. This regional perspective on GDP growth offers valuable insights into the varying economic performance across different parts of the world.

In [ ]:
%%sql
WITH RegionalAvgGDPGrowth AS (
    SELECT
        d.region,
        ROUND(AVG(e.gdp_growth),2) AS average_regional_growth
    FROM
        demographics d
    JOIN
        economy e ON d.country_code = e.country_code
    GROUP BY
        d.region
)
SELECT
    rag.region,
    rag.average_regional_growth,
    (SELECT ROUND(AVG(gdp_growth),2) FROM economy) AS global_average_growth,
    CASE
        WHEN rag.average_regional_growth > (SELECT AVG(gdp_growth) FROM economy) THEN 'Above Global Average'
        ELSE 'Below Global Average'
    END AS growth_compared_to_global
FROM
    RegionalAvgGDPGrowth rag
ORDER BY
    rag.average_regional_growth DESC;


 * sqlite:///world_data_v4.db
Done.


region,average_regional_growth,global_average_growth,growth_compared_to_global
Northern America,1045214.0,96251.3,Above Global Average
Eastern Asia,749629.0,96251.3,Above Global Average
Western Europe,480840.22,96251.3,Above Global Average
Oceania,190764.67,96251.3,Above Global Average
Eastern Europe,128602.2,96251.3,Above Global Average
South-Eastern Asia,128173.64,96251.3,Above Global Average
Northern Europe,116866.0,96251.3,Above Global Average
Southern Europe,71540.07,96251.3,Below Global Average
Western Asia,70926.16,96251.3,Below Global Average
Central America,63758.88,96251.3,Below Global Average


The analysis of average regional GDP compared to the global average reveals a notable disparity in economic output across different parts of the world. Several major regions, including Northern America, Eastern Asia, and Western Europe, demonstrate average total GDP values significantly higher than the global average, indicating stronger overall economic activity. Conversely, a larger number of regions, such as Southern Asia, Africa (across various sub-regions), and parts of Latin America and the Pacific, exhibit average GDP figures below the global benchmark, suggesting lower overall economic output within these areas as represented in the dataset. This broad regional comparison highlights the uneven distribution of economic activity globally.

### Examining Extremes in Life Expectancy, Infant Mortality, and Fertility

This segment of the analysis focused on identifying countries at the extremes of key health and demographic indicators. By examining the highest and lowest values for life expectancy in both genders, the country with the highest infant mortality rate, and the nation with the lowest fertility rate within the dataset, the aim was to highlight significant disparities in these critical aspects of global development. The findings from these targeted queries provide a stark illustration of the diverse challenges and successes experienced by different countries in areas concerning health and population dynamics.

In [ ]:
%%sql
SELECT
    d.name AS country_name,
    ls.life_expectancy_female,
    ls.life_expectancy_male
FROM
    demographics d
JOIN
    lifespan ls ON d.country_code = ls.country_code
ORDER BY
    ls.life_expectancy_female DESC, ls.life_expectancy_male DESC
LIMIT 1; -- Country with highest female and male life expectancy



   sqlite:///world_data_v2.db
   sqlite:///world_data_v3.db
 * sqlite:///world_data_v4.db
Done.


country_name,life_expectancy_female,life_expectancy_male
Switzerland,81.6,85.4


In [ ]:
%%sql
SELECT
    d.name AS country_name,
    ls.life_expectancy_female,
    ls.life_expectancy_male
FROM
    demographics d
JOIN
    lifespan ls ON d.country_code = ls.country_code
ORDER BY
    ls.life_expectancy_female ASC, ls.life_expectancy_male ASC
LIMIT 1; -- Country with lowest female and male life expectancy


   sqlite:///world_data_v2.db
   sqlite:///world_data_v3.db
 * sqlite:///world_data_v4.db
Done.


country_name,life_expectancy_female,life_expectancy_male
Lesotho,50.4,56.8


In [ ]:
%%sql
SELECT
    d.name AS country_name,
    ls.infant_mortality
FROM
    demographics d
JOIN
    lifespan ls ON d.country_code = ls.country_code
ORDER BY
    ls.infant_mortality DESC
LIMIT 1; -- Country with highest infant mortality

   sqlite:///world_data_v2.db
   sqlite:///world_data_v3.db
 * sqlite:///world_data_v4.db
Done.


country_name,infant_mortality
Central African Republic,81.9


In [ ]:
%%sql
SELECT
    d.name AS country_name,
    ls.fertility 
FROM
    demographics d
JOIN
    lifespan ls ON d.country_code = ls.country_code
ORDER BY
    ls.fertility ASC
LIMIT 1; -- Country with lowest fertility rate

   sqlite:///world_data_v2.db
   sqlite:///world_data_v3.db
 * sqlite:///world_data_v4.db
Done.


country_name,fertility
"Korea, Republic Of",1.1


The analysis of extreme values revealed Switzerland as the country with the highest life expectancy for both females (85.4 years) and males (81.6 years) in the dataset, indicating a high standard of healthcare and overall well-being. In contrast, Lesotho exhibited the lowest life expectancy for both genders, with females averaging 50.4 years and males 56.8 years, underscoring significant health challenges. 

The Central African Republic recorded the highest infant mortality rate at a staggering 81.9 deaths per 1,000 live births, pointing to critical issues in infant healthcare and survival.

Finally, the Republic of Korea displayed the lowest fertility rate at just 1.1 births per woman, signaling a potential demographic shift with implications for population growth and aging. These stark contrasts underscore the diverse realities in health and demographic outcomes across the globe.

### Regional Ranking of Average Unemployment Rates

To understand the geographical distribution of unemployment, the analysis calculated the average unemployment rate for each region within the dataset. Subsequently, these regional averages were ranked to identify areas experiencing the highest and lowest levels of unemployment. This ranking provides a comparative perspective on labor market conditions across different regions, highlighting areas where employment opportunities may be more or less prevalent on average.

In [ ]:
%%sql
SELECT
    region,
    average_unemployment_rate,
    RANK() OVER (ORDER BY average_unemployment_rate DESC) AS unemployment_rank
FROM (
    SELECT
        d.region,
        ROUND(AVG(l.unemployment),2) AS average_unemployment_rate
    FROM
        demographics d
    JOIN
        labour l ON d.country_code = l.country_code
    GROUP BY
        d.region
) AS RegionalUnemployment
ORDER BY
    unemployment_rank;



 * sqlite:///world_data_v4.db
Done.


region,average_unemployment_rate,unemployment_rank
Southern Africa,23.0,1
Northern Africa,13.36,2
Micronesia,13.17,3
Southern Europe,10.93,4
Caribbean,10.39,5
Western Asia,8.41,6
South America,7.9,7
Middle Africa,7.65,8
Central Asia,6.48,9
Southern Asia,6.21,10


The ranking of regions by their average unemployment rate reveals significant disparities in labor market conditions globally. Southern Africa emerges with the highest average unemployment rate (23.0%), indicating substantial challenges in employment within this region. Northern Africa (13.36%) and Micronesia (13.17%) also exhibit high average unemployment rates. Conversely, South-Eastern Asia (3.11%) and Melanesia (2.88%) show the lowest average unemployment rates among the regions analyzed, suggesting relatively stronger labor markets on average. This regional comparison underscores the varying economic landscapes and employment opportunities across different parts of the world.

### A Regional Comparison of Male vs. Female Post-Secondary Enrollment

The final stage of our analysis investigated gender dynamics in post-secondary education across different regions. By calculating the average enrollment rates for both females and males in each region and then comparing these averages, we aimed to identify regional trends in gender representation at the tertiary education level. This regional comparison highlights areas where one gender may have a significantly higher average enrollment in post-secondary education compared to the other, or where gender parity is more closely observed.

In [ ]:
%%sql
SELECT
    d.region,
    ROUND(AVG(ed.post_sec_enrollment_female),2) AS average_female_post_sec_enrollment,
    ROUND(AVG(ed.post_sec_enrollment_male),2) AS average_male_post_sec_enrollment,
    CASE
        WHEN ROUND(AVG(ed.post_sec_enrollment_female),2) > ROUND(AVG(ed.post_sec_enrollment_male),2) THEN 'Female Higher'
        WHEN ROUND(AVG(ed.post_sec_enrollment_male),2) > ROUND(AVG(ed.post_sec_enrollment_female),2) THEN 'Male Higher'
        ELSE 'Equal'
    END AS post_sec_enrollment_gender_comparison
FROM
    demographics d
JOIN
    education ed ON d.country_code = ed.country_code
GROUP BY
    d.region
ORDER BY
    d.region;

 * sqlite:///world_data_v4.db
Done.


region,average_female_post_sec_enrollment,average_male_post_sec_enrollment,post_sec_enrollment_gender_comparison
Caribbean,102.67,105.22,Male Higher
Central America,101.03,102.77,Male Higher
Central Asia,100.74,101.4,Male Higher
Eastern Africa,106.96,108.26,Male Higher
Eastern Asia,103.6,104.0,Male Higher
Eastern Europe,96.7,96.78,Male Higher
Melanesia,105.2,107.6,Male Higher
Micronesia,94.9,102.25,Male Higher
Middle Africa,99.73,110.43,Male Higher
Northern Africa,103.65,107.02,Male Higher


The analysis of average post-secondary enrollment rates by gender across different regions reveals a notable trend of male enrollment generally being higher than female enrollment. Across the majority of the regions analyzed, including the Caribbean, Central America, Central Asia, various parts of Africa, Asia, Europe, and the Americas, the average male post-secondary enrollment rate slightly exceeds that of females. There are a few exceptions: Oceania and Southern Europe show a slightly higher average female enrollment. Northern Europe exhibits near gender parity with essentially equal average enrollment rates. This regional comparison suggests that while progress towards gender equality in education has been made, a slight male predominance in post-secondary enrollment persists in many parts of the world according to this dataset.

## Conclusion

In conclusion, this analysis of global development indicators has revealed significant variations and trends across different regions and countries. 

From disparities in economic growth and health outcomes to differences in labor market conditions and post-secondary education enrollment by gender, the insights gleaned from the data underscore the complex and multifaceted nature of global development. 

The application of database management techniques and SQL querying allowed for a structured exploration of these interconnected factors, providing a quantitative basis for understanding the diverse challenges and successes observed worldwide. 

Further research could delve deeper into the underlying causes of these regional and national differences and explore the potential correlations between various indicators in more detail.